In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.mixture import GaussianMixture


## N = 18

In [2]:
# prompt: mount to google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [49]:
import numpy as np

# load data
X = np.load("/content/drive/MyDrive/math-ml/Datasets/Datasets/kryptonite-18-X.npy")
X.shape

(36000, 18)

In [50]:
y = np.load("/content/drive/MyDrive/math-ml/Datasets/Datasets/kryptonite-18-y.npy")
y.shape

(36000,)

## Data Transformation

In [51]:

# Assuming X and y are your feature matrix and labels, respectively
# X is a 2D array where rows are samples and columns are features

# Step 1: Apply GMM Transformation to each feature in X
X_gmm = np.zeros_like(X)

for i in range(X.shape[1]):
    # Fit GMM with 2 components for each feature
    gmm = GaussianMixture(n_components=2, random_state=0)
    gmm.fit(X[:, i].reshape(-1, 1))
    X_gmm[:, i] = gmm.predict(X[:, i].reshape(-1, 1))

# Step 2: Combine the original features with the GMM-labeled features
X_combined = np.concatenate([X, X_gmm], axis=1)

# # Step 3: Split the combined data into training, validation, and test sets
# X_train, X_temp, y_train, y_temp = train_test_split(X_combined, y, test_size=0.8, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# # Print shapes to verify
# print("Shape of X_train:", X_train.shape)
# print("Shape of X_val:", X_val.shape)
# print("Shape of X_test:", X_test.shape)

In [52]:
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, random_split

# Convert numpy arrays to PyTorch tensors
X_tensor = torch.tensor(X_combined, dtype=torch.float)
y_tensor = torch.tensor(y, dtype=torch.float)

# create a TensorDataset
dataset = TensorDataset(X_tensor, y_tensor)

# define split sizes (80% train, 10% validation, 10% test)
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Split the dataset into train, validation, and test
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders for each subset
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# check data loader output
for X_batch, y_batch in tqdm(train_loader):
    print(X_batch.shape, y_batch.shape)  # X_batch is of shape [batch_size, *input_shape]
    break


  0%|          | 0/225 [00:00<?, ?it/s]

torch.Size([128, 36]) torch.Size([128])


In [53]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# build model for training
class MLP(nn.Module):
    def __init__(self, input_size=36, hidden_size=128, output_size=1):
        super(MLP, self).__init__()

        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, hidden_size*2)
        self.layer3 = nn.Linear(hidden_size*2, hidden_size*2)
        # self.layer4 = nn.Linear(hidden_size*2, hidden_size)
        self.layer5 = nn.Linear(hidden_size*2, output_size)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        # x = F.relu(self.layer4(x))
        x = torch.sigmoid(self.layer5(x))  # sigmoid activation for binary output
        return x


In [54]:
# initialize the model, loss function, and optimizer
model = MLP(input_size=36, hidden_size=128, output_size=1)
loss_fn = nn.BCELoss()  # binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)

# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Move the model to the GPU
model.to(device)

Using device: cpu


MLP(
  (layer1): Linear(in_features=36, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=256, bias=True)
  (layer3): Linear(in_features=256, out_features=256, bias=True)
  (layer5): Linear(in_features=256, out_features=1, bias=True)
)

In [55]:
torch.manual_seed(42)

# set the number of epochs
epochs = 60
best_acc = 0

for epoch in range(epochs):
    """ Training """
    model.train()

    # forward pass
    correct = 0
    total = 0
    train_loss = 0
    for X_batch, y_batch in train_loader:

        # Move data to the GPU
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        output = model(X_batch).squeeze()
        y_preds = torch.round(output)

        correct += torch.eq(y_preds, y_batch).sum().item()
        total += len(y_batch)

        loss = loss_fn(output, y_batch)
        train_loss += loss.item()
        # zero the optimizer
        optimizer.zero_grad()
        # backpropagattion
        loss.backward()
        # GD
        optimizer.step()

    scheduler.step()
    train_acc = (correct / total) * 100

    """ Testing """
    model.eval()
    correct = 0
    total = 0
    val_loss = 0
    with torch.inference_mode():
        for X_batch, y_batch in val_loader:
            output = model(X_batch).squeeze()
            y_preds = torch.round(output)

            correct += torch.eq(y_preds, y_batch).sum().item()
            total += len(y_batch)

            loss = loss_fn(output, y_batch)
            val_loss += loss.item()

        val_acc = (correct / total) * 100
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "n-15best.pth")

    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Train Loss: {train_loss:.5f} | Acc: {train_acc:.2f}% | Learning Rate: {scheduler.get_last_lr()[0]:.7f} | Test loss: {val_loss:.5f} | Test Acc: {val_acc:.2f}%")

Epoch: 0 | Train Loss: 156.04414 | Acc: 49.90% | Learning Rate: 0.0010000 | Test loss: 20.09862 | Test Acc: 50.42%
Epoch: 10 | Train Loss: 37.82350 | Acc: 95.18% | Learning Rate: 0.0010000 | Test loss: 5.23209 | Test Acc: 94.22%
Epoch: 20 | Train Loss: 31.45525 | Acc: 96.54% | Learning Rate: 0.0010000 | Test loss: 4.94523 | Test Acc: 96.33%
Epoch: 30 | Train Loss: 28.62321 | Acc: 96.78% | Learning Rate: 0.0010000 | Test loss: 4.63058 | Test Acc: 96.53%
Epoch: 40 | Train Loss: 25.70728 | Acc: 96.91% | Learning Rate: 0.0001000 | Test loss: 4.55854 | Test Acc: 96.58%
Epoch: 50 | Train Loss: 24.69711 | Acc: 96.96% | Learning Rate: 0.0001000 | Test loss: 4.64621 | Test Acc: 96.56%


In [ ]:
model = MLP(input_size=15, hidden_size=128, output_size=1)
model.load_state_dict(torch.load("n-15best.pth"))
model.eval()

correct = 0
total = 0
val_loss = 0
with torch.inference_mode():
    for X_batch, y_batch in tqdm(val_loader):
        output = model(X_batch).squeeze()
        y_preds = torch.round(output)

        correct += torch.eq(y_preds, y_batch).sum().item()
        total += len(y_batch)

        loss = loss_fn(output, y_batch)
        val_loss += loss.item()

    val_acc = (correct / total) * 100

print(f"Validation Loss: {val_loss:.5f} | Validation Acc: {val_acc:.2f}%")